## Loading packages

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from google.colab import files
!pip install xlrd
import xlrd

    100% |████████████████████████████████| 112kB 4.2MB/s 


## Titanic dataset

In [0]:
data = files.urllib.request.urlretrieve('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls', filename=None)[0]
df = pd.read_excel(data, encoding = 'utf-8')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.2+ KB


### Data preprocessing

array([[1, 1, u'Allen, Miss. Elisabeth Walton', ..., 2, nan,
        u'St Louis, MO'],
       [1, 1, u'Allison, Master. Hudson Trevor', ..., u'11', nan,
        u'Montreal, PQ / Chesterville, ON'],
       [1, 0, u'Allison, Miss. Helen Loraine', ..., nan, nan,
        u'Montreal, PQ / Chesterville, ON'],
       ...,
       [3, 0, u'Zakarian, Mr. Mapriededer', ..., nan, 304.0, nan],
       [3, 0, u'Zakarian, Mr. Ortin', ..., nan, nan, nan],
       [3, 0, u'Zimmerman, Mr. Leo', ..., nan, nan, nan]], dtype=object)